In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.{DataFrame,Column}
import org.json4s._
import org.json4s.jackson.JsonMethods._

In [3]:
val spark = builder.master("yarn-mode").appName("Data Preprocessing").getOrCreate;

spark = org.apache.spark.sql.SparkSession@27ab3a2c


Spark local-1558076652834: Some(http://ip-172-31-38-146.ec2.internal:4044)

In [4]:
def getNameCollection = udf((entry: String) => {
	// ubah None jadi null
 	val clearNone = entry.replaceAll("None", "null")

 	// jika value json stringnya ' maka harus digantikan dengan "
 	val cleanq = clearNone.replaceAll("'", "\"")


 	// ambil value
 	try {
 		val parser = parse(cleanq)
 		 def scalaFiles = for {
         JObject(child) <- parser
         JField("name", JString(name))  <- child
       } yield name
    	scalaFiles
 	} catch {
 		 case e: Exception => List(null)
 	}

     })

getNameCollection: org.apache.spark.sql.expressions.UserDefinedFunction


In [ ]:
val df = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("quote","\"").option("delimiter",";").load("train2.csv")

In [ ]:
// getdatas
val newdf = df.select("id","belongs_to_collection","genres","spoken_languages","Keywords","cast","crew","title","tagline","Revenue").
withColumn("collection",
		when(
              col("belongs_to_collection").isNotNull,
              getNameCollection(new Column("belongs_to_collection"))
            )
	).
withColumn("newgenres",
		when(
              col("genres").isNotNull,
              getNameCollection(new Column("genres"))
            )
	).
withColumn("newspoken",
		when(
              col("spoken_languages").isNotNull,
              getNameCollection(new Column("spoken_languages"))
            )
	).
withColumn("newkeywords",
		when(
              col("Keywords").isNotNull,
              getNameCollection(new Column("Keywords"))
            )
	).
withColumn("newcast",
		when(
              col("cast").isNotNull,
              getNameCollection(new Column("cast"))
            )
	).
withColumn("newcrew",
		when(
              col("crew").isNotNull,
              getNameCollection(new Column("crew"))
            )
	).
select("id","collection","newgenres","newspoken","newkeywords","newcast","newcrew","title","tagline","Revenue").
withColumnRenamed("newgenres","genres").
withColumnRenamed("newkeywords","keywords").
withColumnRenamed("newcast","cast").
withColumnRenamed("newcrew","crew").
withColumnRenamed("newspoken","spoken")

In [ ]:
val dfclean = (
    newdf.withColumn("tempcollection",explode_outer(col("collection")))
    .withColumn("tempgenres",explode_outer(col("genres")))
    .withColumn("tempspoken",explode_outer(col("spoken")))
    .withColumn("tempkeywords",explode_outer(col("keywords")))
    .withColumn("tempcast",explode_outer(col("cast")))
    .withColumn("tempcrew",explode_outer(col("crew")))
    .select("id","tempcollection","tempgenres","tempspoken","tempkeywords","tempcast","tempcrew","tagline","Revenue")
    .withColumnRenamed("tempcollection","collection")
    .withColumnRenamed("tempgenres","genres")
    .withColumnRenamed("tempspoken","spoken")
    .withColumnRenamed("tempkeywords","keywords")
    .withColumnRenamed("tempcast","cast")
    .withColumnRenamed("tempcrew","crew")
    )

In [ ]:
output dfclean:
+---+--------------------+------+-------+-----------+--------------------+----+--------------------+--------+
| id|          collection|genres| spoken|   keywords|                cast|crew|             tagline| Revenue|
+---+--------------------+------+-------+-----------+--------------------+----+--------------------+--------+
|  1|Hot Tub Time Mach...|Comedy|English|time travel|         Rob Corddry|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|      Craig Robinson|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|          Clark Duke|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|          Adam Scott|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|         Chevy Chase|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|      Gillian Jacobs|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|        Bianca Haase|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|      Collette Wolfe|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|     Kumail Nanjiani|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|      Kellee Stewart|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|          Josh Heald|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|    Gretchen Koerner|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|           Lisa Loeb|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|    Jessica Williams|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|        Bruce Buffer|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|Mariana Paola Vic...|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|    Christian Slater|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|         Jason Jones|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|       Olivia Jordan|null|The Laws of Space...|12314651|
|  1|Hot Tub Time Mach...|Comedy|English|time travel|    Christine Bently|null|The Laws of Space...|12314651|
+---+--------------------+------+-------+-----------+--------------------+----+--------------------+--------+